In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
import numpy as np
%pprint

notebook_path = get_notebook_path()
print(notebook_path)

s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\Miscellaneous\ipynb\Untitled.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'exit', 'get_classifier', 'get_data_structs_df', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_notebook_path', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'pickle', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_starting_with', 'quit', 're', 'remove_empty_folders', 's', 'similar', 'subprocess', 'sys', 'time', 'urllib']

In [2]:

!start %windir%\explorer.exe "{os.path.abspath(s.data_folder)}"

In [3]:

import json

json_dir = os.path.join(s.data_folder, 'json')
file_path = os.path.join(json_dir, 'full-analysis2.json')
with open(file_path) as fh:
    json_data = json.loads(fh.read())
print(['json_data.{}'.format(fn) for fn in dir(json_data) if not fn.startswith('_')])

['json_data.clear', 'json_data.copy', 'json_data.fromkeys', 'json_data.get', 'json_data.items', 'json_data.keys', 'json_data.pop', 'json_data.popitem', 'json_data.setdefault', 'json_data.update', 'json_data.values']


In [6]:

twelve_list = json_data['12']

In [9]:

for user_dict in twelve_list:
    screen_name = user_dict['screen_name']
    twitter_url = 'https://twitter.com/{}'.format(screen_name)
    !"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {twitter_url}

Opening in existing browser session.


C:\Program Files (x86)\Google\Chrome\Application\chrome.exe: invalid option -- `-u'
Try 'chrome.exe --help' for more information.
[0319/160432.156:ERROR:registration_protocol_win.cc(84)] TransactNamedPipe: The pipe has been ended. (0x6D)
[203740:200932:0319/160432.459:ERROR:cache_util_win.cc(21)] Unable to move the cache: Access is denied. (0x5)
[203740:200932:0319/160432.459:ERROR:cache_util.cc(135)] Unable to move cache folder C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\GPUCache to C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\old_GPUCache_000
[203740:200932:0319/160432.459:ERROR:disk_cache.cc(184)] Unable to create cache
[203740:200932:0319/160432.459:ERROR:shader_disk_cache.cc(606)] Shader Cache Creation failed: -2


Opening in existing browser session.


C:\Program Files (x86)\Google\Chrome\Application\chrome.exe: invalid option -- `-u'
Try 'chrome.exe --help' for more information.
[0319/160433.887:ERROR:registration_protocol_win.cc(84)] TransactNamedPipe: The pipe has been ended. (0x6D)
[3812:2676:0319/160434.143:ERROR:cache_util_win.cc(21)] Unable to move the cache: Access is denied. (0x5)
[3812:2676:0319/160434.144:ERROR:cache_util.cc(135)] Unable to move cache folder C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\GPUCache to C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\old_GPUCache_000
[3812:2676:0319/160434.144:ERROR:disk_cache.cc(184)] Unable to create cache
[3812:2676:0319/160434.144:ERROR:shader_disk_cache.cc(606)] Shader Cache Creation failed: -2


Opening in existing browser session.


...

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe: invalid option -- `-u'
Try 'chrome.exe --help' for more information.
[0319/160446.236:ERROR:registration_protocol_win.cc(84)] TransactNamedPipe: The pipe has been ended. (0x6D)
[186968:193016:0319/160446.567:ERROR:cache_util_win.cc(21)] Unable to move the cache: Access is denied. (0x5)
[186968:193016:0319/160446.567:ERROR:cache_util.cc(135)] Unable to move cache folder C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\GPUCache to C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\old_GPUCache_000
[186968:193016:0319/160446.567:ERROR:disk_cache.cc(184)] Unable to create cache
[186968:193016:0319/160446.567:ERROR:shader_disk_cache.cc(606)] Shader Cache Creation failed: -2


Opening in existing browser session.


C:\Program Files (x86)\Google\Chrome\Application\chrome.exe: invalid option -- `-u'
Try 'chrome.exe --help' for more information.
[0319/160448.452:ERROR:registration_protocol_win.cc(84)] TransactNamedPipe: The pipe has been ended. (0x6D)
[4504:203952:0319/160448.759:ERROR:cache_util_win.cc(21)] Unable to move the cache: Access is denied. (0x5)
[4504:203952:0319/160448.759:ERROR:cache_util.cc(135)] Unable to move cache folder C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\GPUCache to C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\old_GPUCache_000
[4504:203952:0319/160448.759:ERROR:disk_cache.cc(184)] Unable to create cache
[4504:203952:0319/160448.759:ERROR:shader_disk_cache.cc(606)] Shader Cache Creation failed: -2


Opening in existing browser session.


C:\Program Files (x86)\Google\Chrome\Application\chrome.exe: invalid option -- `-u'
Try 'chrome.exe --help' for more information.
[0319/160450.693:ERROR:registration_protocol_win.cc(84)] TransactNamedPipe: The pipe has been ended. (0x6D)
[190816:200952:0319/160451.007:ERROR:cache_util_win.cc(21)] Unable to move the cache: Access is denied. (0x5)
[190816:200952:0319/160451.007:ERROR:cache_util.cc(135)] Unable to move cache folder C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\GPUCache to C:\Users\577342\AppData\Local\Google\Chrome\User Data\ShaderCache\old_GPUCache_000
[190816:200952:0319/160451.007:ERROR:disk_cache.cc(184)] Unable to create cache
[190816:200952:0319/160451.007:ERROR:shader_disk_cache.cc(606)] Shader Cache Creation failed: -2


In [8]:

twelve_list[0]

{'short_id': '25396', 'id': 799645, 'communities': 12, 'degree': 687, 'degree_c': 0.41389728096676737, 'katz_c': 0.6809489551399306, 'screen_name': 'kenyatta', 'followers_count': 7459, 'friends_count': 3316, 'verified': False, 'statuses_count': 16429, 'created_at': 'Wed Feb 28 00:21:10 +0000 2007', 'description': 'aka @ea1co @knowyourmeme @urbanjustice @ellecheese @eyebeamnyc\nA++++! Great communication, fast shipping! Would do business with again!'}